# STDP例：生物学的学習

**説明：** ニューロモーフィックニューラルネットワークで使用される生物学的学習メカニズムSTDP（スパイクタイミング依存可塑性）に関するインタラクティブチュートリアル。ニューロンが時間的相関を自動的に学習する方法を示します。

**著者：** Mauro Risonho de Paula Assumpção
**作成日：** 2025年12月5日
**ライセンス：** MITライセンス
**開発：** 人間 + AI支援開発（Claude Sonnet 4.5、Gemini 3 Pro Preview）

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
# Brian2ライブラリがまだインストールされていない場合はインストールする
try:
    import brian2
except ImportError:
    !pip install brian2
    import brian2

# ワイルドカードの代わりにbrian2から特定のインポート
from brian2 import (
    ms, mV, Hz, second,
    NeuronGroup, Synapses, SpikeMonitor, StateMonitor,
    SpikeGeneratorGroup, Network,
    defaultclock, run, device, start_scope,
    clip, prefs
)

# numpyを使用するように設定（ヘッダーが見つからない場合のC++コンパイルエラーを回避）
prefs.codegen.target = "numpy"

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

print("✓ インポート完了！")

# STDP: スパイクタイミング依存可塑性

**説明：** ニューロモーフィックニューラルネットワークで使用される生物学的学習メカニズムSTDP（スパイクタイミング依存可塑性）に関するインタラクティブチュートリアル。ニューロンが時間的相関を自動的に学習する方法を示します。

**著者：** Mauro Risonho de Paula Assumpção
**作成日：** 2025年12月5日
**ライセンス：** MITライセンス
**開発：** 人間 + AI支援開発（Claude Sonnet 4.5、Gemini 3 Pro Preview）

---

このノートブックは、ニューロモーフィックニューラルネットワークで使用される生物学的学習メカニズム**STDP**を探求します。

## STDPとは？

STDP（スパイクタイミング依存可塑性）は、生物学的ニューロンに触発された**教師なし**学習ルールです：

- **シナプス前ニューロンがシナプス後ニューロンの前に発火する場合** → **増強**（重み↑）
- **シナプス前ニューロンがシナプス後ニューロンの後に発火する場合** → **減弱**（重み↓）

これにより、ネットワークは明示的なラベルなしで**時間的因果関係**を学習できます。

## 1. セットアップとインポート

## 1. 古典的なSTDP曲線

スパイク間の時間差に応じて重みの変化がどのように依存するかを視覚化します。

In [ ]:
# STDPパラメータ
tau_pre = 20.0 # ms - シナプス前時定数
tau_post = 20.0 # ms - シナプス後時定数
A_pre = 0.01 # 増強振幅
A_post = -0.012 # 減弱振幅

# デルタt（時間差）
dt_range = np.linspace(-100, 100, 500) # ms

# 重みの変化を計算
def stdp_weight_change(dt, tau_pre, tau_post, A_pre, A_post):
    """
    STDPによる重みの変化を計算します。
    dt = t_post - t_pre
    """
    if dt > 0: # シナプス後がシナプス前の後 → 増強
        return A_pre * np.exp(-dt / tau_pre)
    else: # シナプス後がシナプス前の前 → 減弱
        return A_post * np.exp(dt / tau_post)

weight_changes = np.array([stdp_weight_change(dt, tau_pre, tau_post, A_pre, A_post)
                           for dt in dt_range])

# STDP曲線をプロット
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(dt_range, weight_changes, linewidth=3, color='purple')
ax.axhline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax.axvline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)

# 領域を注釈
ax.fill_between(dt_range[dt_range > 0], 0, weight_changes[dt_range > 0],
                alpha=0.2, color='green', label='増強（LTP）')
ax.fill_between(dt_range[dt_range < 0], 0, weight_changes[dt_range < 0],
                alpha=0.2, color='red', label='減弱（LTD）')

ax.set_xlabel('Δt = t_post - t_pre (ms)', fontsize=12)
ax.set_ylabel('重みの変化（Δw）', fontsize=12)
ax.set_title('STDP曲線：スパイクタイミング依存可塑性', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# 注釈を追加
ax.annotate('シナプス前 → シナプス後\n（因果的）', xy=(20, 0.008), fontsize=10,
            ha='center', color='green', fontweight='bold')
ax.annotate('シナプス後 → シナプス前\n（反因果的）', xy=(-20, -0.009), fontsize=10,
            ha='center', color='red', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✓ 解釈：")
print("  - Δt > 0: シナプス前ニューロンが先に発火 → 増強（接続を強化）")
print("  - Δt < 0: シナプス前ニューロンが後に発火 → 減弱（接続を弱化）")
print("  - 効果は|Δt|に応じて指数関数的に減衰")

## 2. Brian2によるSTDPシミュレーション

STDPで接続された2つのニューロンをシミュレートし、重みの進化を観察します。

In [ ]:
start_scope()

# シミュレーションパラメータ
duration = 100*ms  # type: ignore[operator]
defaultclock.dt = 0.1*ms  # type: ignore[operator]

print("⚙️ STDPシミュレーションを設定しています...")
print(f"期間: {duration}")
print(f"時間ステップ: {defaultclock.dt}\n")

# LIFニューロン
tau_m = 10*ms  # type: ignore[operator]
tau_syn = 5*ms  # type: ignore[operator] # シナプス時定数
v_rest = -70*mV
v_thresh = -50*mV
v_reset = -70*mV

# シナプス減衰を追加（dI_syn/dt）
eqs_post = '''
dv/dt = (v_rest - v + I_syn) / tau_m : volt
dI_syn/dt = -I_syn / tau_syn : volt
'''

# ニューロンを作成
neuron_pre = SpikeGeneratorGroup(1, [0], [10]*ms)  # type: ignore[operator]
neuron_post = NeuronGroup(1, eqs_post, threshold='v > v_thresh',
                          reset='v = v_reset', method='euler')
neuron_post.v = v_rest
neuron_post.I_syn = 0*mV

# STDPパラメータ
tau_pre_stdp = 20*ms  # type: ignore[operator]
tau_post_stdp = 20*ms  # type: ignore[operator]
A_pre_stdp = 0.01
A_post_stdp = -0.012
w_max = 1.0
w_min = 0.0

synapse_model = '''
w : 1
dApre/dt = -Apre / tau_pre_stdp : 1 (event-driven)
dApost/dt = -Apost / tau_post_stdp : 1 (event-driven)
'''

# 発火を確実にするためにシナプスゲインを増加（w * 60*mV）
on_pre_stdp = '''
I_syn_post += w * 60 * mV
Apre += A_pre_stdp
w = clip(w + Apost, w_min, w_max)
'''

on_post_stdp = '''
Apost += A_post_stdp
w = clip(w + Apre, w_min, w_max)
'''

synapse = Synapses(neuron_pre, neuron_post,
                   model=synapse_model,
                   on_pre=on_pre_stdp,
                   on_post=on_post_stdp,
                   method='euler')
synapse.connect(i=0, j=0)
synapse.w = 0.5 # 初期重み

# モニター
mon_pre = SpikeMonitor(neuron_pre)
mon_post = SpikeMonitor(neuron_post)
mon_weight = StateMonitor(synapse, 'w', record=True)
mon_voltage = StateMonitor(neuron_post, 'v', record=True)

# シミュレーションを実行
print("⏳ Brian2シミュレーションを実行しています...")
import time
start_time = time.time()

net = Network(neuron_pre, neuron_post, synapse, mon_pre, mon_post, mon_weight, mon_voltage)
net.run(duration)

sim_time = time.time() - start_time

print(f"✅ シミュレーションが{sim_time:.3f}秒で完了しました！")
print(f"\n結果:")
print(f" シナプス前スパイク: {len(mon_pre.t)}")
print(f" シナプス後スパイク: {len(mon_post.t)}")
print(f" 初期重み: {0.5:.3f}")
print(f" 最終重み: {mon_weight.w[0][-1]:.3f}")
print(f" 変化: {(mon_weight.w[0][-1] - 0.5):.3f} ({(mon_weight.w[0][-1] - 0.5)/0.5*100:+.1f}%)")

In [ ]:
# 結果を視覚化
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# プロット1: スパイク
if len(mon_pre.t) > 0:
    axes[0].eventplot([mon_pre.t/ms], lineoffsets=1, linelengths=0.8,
                      linewidths=2, colors='blue', label='シナプス前')
if len(mon_post.t) > 0:
    axes[0].eventplot([mon_post.t/ms], lineoffsets=0, linelengths=0.8,
                      linewidths=2, colors='red', label='シナプス後')

axes[0].set_ylabel('ニューロン')
axes[0].set_yticks([0, 1])
axes[0].set_yticklabels(['シナプス後', 'シナプス前'])
axes[0].set_title('ラスタープロット：シナプス前・後スパイク', fontsize=12, fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# プロット2: シナプス重みの進化
axes[1].plot(mon_weight.t/ms, mon_weight.w[0], linewidth=2.5, color='purple')
axes[1].axhline(0.5, color='gray', linestyle='--', alpha=0.5, label='初期重み')
axes[1].set_ylabel('シナプス重み（w）')
axes[1].set_title('STDPによるシナプス重みの進化', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# プロット3: シナプス後ニューロンの電圧
axes[2].plot(mon_voltage.t/ms, mon_voltage.v[0]/mV, linewidth=1.5, color='green')
axes[2].axhline(-50, color='red', linestyle='--', alpha=0.7, label='閾値')
axes[2].axhline(-70, color='gray', linestyle='--', alpha=0.5, label='静止')
axes[2].set_xlabel('時間（ms）')
axes[2].set_ylabel('電圧（mV）')
axes[2].set_title('シナプス後膜電位', fontsize=12, fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. 入力パターンを持つSTDP

STDPが繰り返しパターンの時間的相関を学習する方法を実証します。

In [ ]:
start_scope()

# 複数のシナプス前ニューロンをシミュレート
n_pre = 5
n_post = 1
duration = 500*ms  # type: ignore[operator]
defaultclock.dt = 0.1*ms  # type: ignore[operator]

print("⚙️ 複数ニューロンでシミュレーションを設定しています...")
print(f"シナプス前ニューロン: {n_pre}")
print(f"期間: {duration}\n")

# 時間的パターンを生成（一部のニューロンが順番に発火）
spike_pattern = [
    [10, 110, 210, 310, 410], # ニューロン0: 規則的なスパイク
    [15, 115, 215, 315, 415], # ニューロン1: わずかに遅延
    [20, 120, 220, 320, 420], # ニューロン2: より遅延
    [100, 200, 300, 400], # ニューロン3: まばらなスパイク
    [50, 150, 250, 350, 450] # ニューロン4: 異なる位相
]

indices = []
times = []
print("📊 スパイクパターン:")
for neuron_idx, spike_times in enumerate(spike_pattern):
    print(f"  ニューロン {neuron_idx}: {len(spike_times)} スパイク")
    for t in spike_times:
        indices.append(neuron_idx)
        times.append(t)

neuron_pre = SpikeGeneratorGroup(n_pre, indices, times*ms)  # type: ignore[operator]

# シナプス後ニューロン
neuron_post = NeuronGroup(n_post, eqs_post, threshold='v > v_thresh',
                          reset='v = v_reset', method='euler')
neuron_post.v = v_rest

# STDPを持つシナプス
synapse = Synapses(neuron_pre, neuron_post,
                   model=synapse_model,
                   on_pre=on_pre_stdp,
                   on_post=on_post_stdp,
                   method='euler')
synapse.connect() # すべて接続
synapse.w = 'rand() * 0.3 + 0.2' # ランダム初期重み [0.2, 0.5]

# モニター
mon_pre = SpikeMonitor(neuron_pre)
mon_post = SpikeMonitor(neuron_post)
mon_weight = StateMonitor(synapse, 'w', record=True)

# 初期重みを保存
initial_weights = np.array(synapse.w).copy()

# 実行
print("\n⏳ 時間的パターンのシミュレーションを実行しています...")
start_time = time.time()

net = Network(neuron_pre, neuron_post, synapse, mon_pre, mon_post, mon_weight)
net.run(duration)

sim_time = time.time() - start_time
final_weights = np.array(synapse.w).copy()

print(f"✅ シミュレーションが{sim_time:.3f}秒で完了しました！")
print(f"\n📊 シナプス重みの分析:")
for i in range(n_pre):
    delta = final_weights[i] - initial_weights[i]
    percentage = (delta / initial_weights[i]) * 100 if initial_weights[i] > 0 else 0
    print(f"  ニューロン {i}: {initial_weights[i]:.3f} → {final_weights[i]:.3f} (Δ = {delta:+.3f}, {percentage:+.1f}%)")

In [ ]:
# 重みの進化を視覚化
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# プロット1: 重みの時間的進化
for i in range(n_pre):
 axes[0].plot(mon_weight.t/ms, mon_weight.w[i], label=f'シナプス {i}', linewidth=2)

axes[0].set_xlabel('時間（ms）')
axes[0].set_ylabel('シナプス重み')
axes[0].set_title('STDPによるシナプス重みの時間的進化', fontsize=12, fontweight='bold')
axes[0].legend(loc='best')
axes[0].grid(True, alpha=0.3)

# プロット2: 前後比較
x_pos = np.arange(n_pre)
width = 0.35

axes[1].bar(x_pos - width/2, initial_weights, width, label='初期', alpha=0.7, color='lightblue')
axes[1].bar(x_pos + width/2, final_weights, width, label='最終', alpha=0.7, color='darkblue')

axes[1].set_xlabel('シナプス前ニューロン')
axes[1].set_ylabel('シナプス重み')
axes[1].set_title('比較：初期 vs 最終重み', fontsize=12, fontweight='bold')
axes[1].set_xticks(x_pos)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n💡 解釈：")
print("  - シナプス後ニューロンの前に一貫して発火するニューロンは強化されます")
print("  - タイミングが一貫していないニューロンは重みが減少します")
print("  - ネットワークは時間的相関を自動的に学習します！")

## 4. 不正検出への応用

STDPは不正検出にどのように役立つのでしょうか？

### シナリオ1：正常な時間的シーケンス

**正当な取引：**
1. アプリへのログイン（t=0ms）
2. 残高への移動（t=500ms）
3. 既知の受取人の選択（t=2000ms）
4. 支払い確認（t=3000ms）

**STDPが学習：**
- 期待される因果シーケンス
- 正常な時間間隔
- 正当な行動を表す接続を強化

### シナリオ2：異常なシーケンス（不正）

**不正な取引：**
1. アプリへのログイン（t=0ms）
2. ナビゲーションなしで即座の送金（t=50ms）
3. 新しい受取人への高額（t=100ms）
4. 一貫性のない地理的位置（t=150ms）

**STDPが検出：**
- 異常な時間的パターン
- トレーニング中に強化されていないシーケンス
- 「不正」ニューロンの高い活性化

### STDPの利点：

1. **教師なし学習**：最初は明示的なラベルが不要
2. **継続的な適応**：新しい不正パターンを自動的に学習
3. **時間的感度**：イベントシーケンスの異常を検出
4. **効率性**：局所的な重み更新（逆伝播なし）
5. **生物学的妥当性**：人間の脳に触発されている

## 5. 結論

### 不正検出におけるSTDP

**メカニズム：**
- 取引特徴間の時間的相関を学習
- 頻繁な正当なパターンを強化
- 時間的シーケンスの逸脱を検出

**実用的な応用：**
1. **行動分析**：モバイルバンキングでのアクションのシーケンス
2. **速度検出**：不可能な取引（例：数分以内の異なる都市での購入）
3. **使用パターン**：スケジュール、頻度、典型的な値
4. **疑わしいナビゲーション**：非定型のページシーケンス

**従来の方法との比較：**

| 特徴 | STDP/SNN | DNN/LSTM |
|----------------|----------|----------|
| 時間的処理 | ネイティブ | エミュレート |
| 教師 | 不要 | 必要 |
| レイテンシ | 超低（〜ms） | 高（〜100ms） |
| エネルギー消費 | 非常に低い | 高い |
| オンライン適応 | はい | 困難 |
| 専用ハードウェア | はい（Loihi、TrueNorth） | GPU |

### 今後の方向性

- 専用ニューロモーフィックチップ（Intel Loihi 2、IBM NorthPole）
- STDP + 報酬変調（人工ドーパミン）
- STDPを用いた連合学習
- 説明可能性：学習した重みの視覚化

---

**著者：** Mauro Risonho de Paula Assumpção
**プロジェクト：** 銀行サイバーセキュリティのためのニューロモーフィックコンピューティング